$ python3 gpt2-generate.py --model gpt2ja-medium --num_generate 1

In [1]:
#!pip install jaconv

In [2]:
model_path = r"E:\CloudSource\CloudSource_share\tharhtetsan\testing\gpt2-japanes\gpt2-japanese-master\gpt2ja-medium"

In [3]:
import json
import os
import numpy as np
import tensorflow.compat.v1 as tf
import argparse
from tqdm import tqdm
from sampling import sample_sequence
from  encode_bpe  import  BPEEncoder_ja

In [4]:
from model import HParams as HParams

In [5]:
with open('ja-bpe.txt', encoding='utf-8') as f:
    bpe = f.read().split('\n')

with open('emoji.json', encoding='utf-8') as f:
    emoji = json.loads(f.read())

In [6]:
enc  =  BPEEncoder_ja ( bpe , emoji )
n_vocab  =  len ( enc )

In [7]:
hparams = HParams(**{
      "n_vocab": n_vocab,
      "n_ctx": 1024,
      "n_embd": 1024,
      "n_head": 16,
      "n_layer": 24
    })

In [8]:
length=hparams.n_ctx // 2
temperature=1
top_k=40
top_p=0
min_length=max(0,0)
max_length = 500

context = "『GPT-2』は、入力された文章の続きを作成するAIです。このデモンストレーションでは、このエリアに入力された文章の続きとなる文章を生成します。このエリアに、続きを生成したい文章を入力して、ページの下にある「Generate Japanese Text\!」ボタンをクリックしてください。"

In [12]:
def generate_one(sess, output):
    generated = ''
    pre_text = context if len(str(context))>0 else '<|endoftext|>'
    while True:
        context_tokens = enc.encode(pre_text)
        if len(context_tokens) > length:
            context_tokens = context_tokens[-length:]
        out = sess.run(output, feed_dict={
            context: [context_tokens]
        })[:,len(context_tokens):]
        swd = enc.decode(out[0])
        last  =  False
        if '<|endoftext|>' in swd:
            swd = swd.split('<|endoftext|>')[0]
            last = True
        if len(swd) > 0:
            generated += swd

        if last or len(generated) > max_length:
            if len(generated) > 0:
                print("generated : ",generated)
                return generated[:max_length]
        else:
            pre_text = generated[-256:]
        

In [13]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"

In [ ]:
with tf.Session(config=config,graph=tf.Graph()) as sess:
    context = tf.placeholder(tf.int32, [1, None])
    output = sample_sequence(
        hparams=hparams, length=length,
        min_length=min_length, context=context,
        batch_size=1,
        temperature=temperature, top_k=top_k, top_p=top_p
    )
    
    saver = tf.train.Saver()
    print(model_path)
    ckpt = tf.train.latest_checkpoint(model_path)
    saver.restore(sess, ckpt)
    
    num_generate = 1
    for i in range(num_generate):
        print(generate_one(sess, output))
        if i < num_generate-1:
            print('========')
    
    

E:\CloudSource\CloudSource_share\tharhtetsan\testing\gpt2-japanes\gpt2-japanese-master\gpt2ja-medium
INFO:tensorflow:Restoring parameters from E:\CloudSource\CloudSource_share\tharhtetsan\testing\gpt2-japanes\gpt2-japanese-master\gpt2ja-medium\model-10410000
